In [4]:
import rpy2.robjects as robjects




r_script = '''
---
title: "Measures Pre-Processing Pipeline"
output: html_notebook
---

```{r User Definitions}
# Define Name for Measure
Measure_Name <- "Example"

# Set Year For Measure
Update_Year <- "2020"

# Specify Where File Is For Pre-Processing
File_Location <- "https://publisher.ethnicity-facts-figures.service.gov.uk/health/alcohol-smoking-and-drug-use/adult-smokers/4.0/downloads/cigarette-smoking-among-adults.csv"

# Specify Grouping Variables 
Grouping_Vars <- c("Ethnicity", 
                   "Geography", 
                   "Age",
                   "Gender")

# Specify Numeric Variables 
Numeric_Vars <- c("Value",
                  "Samplesize",
                  "Lower 95% C.I.", 
                  "Upper 95% C.I.")

# Specify Variable Names With Time Data
Time_Vars <- c("Time")

# Specify What To Replace Hyphens "-" With In Data (Sometimes ?, !, c, or N/A or 0)
hyphen_replacement <- "0"

```

```{r Check and Install then Load Packages}
package_list <- c("data.table", "ggplot2", "purrr", "tibble", "dplyr",
                  "tidyr", "stringr", "readr", "forcats", "tidylog",
                  "reactable", "janitor", "beepr","tinter", "lubridate", 
                  "htmlwidgets", "curl")

# Check if packages installed, and then install if needed...
for (i in 1:length(package_list)){
  if (length(find.package(package_list[i], quiet=TRUE)) == 0){
    print(paste("Installing'", package_list[i], "..."))
    install.packages(package_list[i])
  } # ...then load packages
  library(package_list[i], character.only = TRUE)
}
```

```{r Setup Directories}
# Test if directories exists, and if not, create them...

directory_list <- c("/Output",  "/Output/(1.) Pre-processed files",
                    "/Output/(2.) Processed files", 
                    "/Output/(3.) Post-processed files",  
                    "/Output/(4.) EFF-tables-charts",  "/Output/(5.) Misc")

for (i in 1:length(directory_list)){
  output_dir <- paste0(getwd(), directory_list[i])
  if (!dir.exists(output_dir)) {dir.create(output_dir)}
}
```

## Load data
```{r Load Data}
# Read Live Data
PreP_Data <- fread(File_Location)

# Check Column Names
colnames(PreP_Data)
```

## Capitalise all column names in Title Case
```{r Capitalise Column Names}
# Rename column names to Title Font
colnames(PreP_Data) <- str_to_title(colnames(PreP_Data))
Grouping_Vars <- str_to_title(Grouping_Vars)
Numeric_Vars <- str_to_title(Numeric_Vars)

# Correct Title Case Exceptions For Columns... 
colnames(PreP_Data) <- str_replace_all(colnames(PreP_Data), pattern = "C.i.", replacement = "C.I.")
Numeric_Vars <-  str_replace_all(Numeric_Vars, pattern = "C.i.", replacement = "C.I.")

colnames(PreP_Data) <- str_replace_all(colnames(PreP_Data), pattern = "Ns Sec", replacement = "NS SEC")
Grouping_Vars <-  str_replace_all(Grouping_Vars, pattern = "Ns Sec", replacement = "NS SEC")

colnames(PreP_Data) <- str_replace_all(colnames(PreP_Data), pattern = "Ns-Sec", replacement = "NS-SEC")
Grouping_Vars <-  str_replace_all(Grouping_Vars, pattern = "Ns-Sec", replacement = "NS-SEC")

colnames(PreP_Data) <- str_replace_all(colnames(PreP_Data), pattern = "_year", replacement = "_Year")
Grouping_Vars <-  str_replace_all(Grouping_Vars, pattern =  "_year", replacement = "_Year")

# If Data Has Column Called 'Year', Rename to Time Instead
if ("Year" %in% colnames(PreP_Data)){
  PreP_Data <- PreP_Data %>% 
    rename("Time" = "Year")
}

# Trim White Space from Column Names
colnames(PreP_Data) <- str_squish(colnames(PreP_Data))

# Check Column Names - Again
colnames(PreP_Data)
```

## Capitalise all values within Grouping Variables into Title Case
```{r Captilisation within Columns}
# If No Update Year is supplied - generate current year
if (is.null(Update_Year)){
  Update_Year <- lubridate::year(Sys.Date())
}

# Check and see if data has more than 1 measure category
if (length(unique(PreP_Data$Measure)) > 1){
  Grouping_Vars <- append("Measure", Grouping_Vars)
}

# Go to each grouping column defined above and apply title case to text within (this is not redoing column name)
PreP_Data <- PreP_Data %>% 
  mutate_at(Grouping_Vars, ~str_to_title(.)) %>%   
  mutate_at(Grouping_Vars, ~str_squish(.)) %>%  
  # Correct Title Case Exceptions For Groups
  mutate_at(Grouping_Vars, ~str_replace_all(., pattern = "Ns Sec", replacement = "NS SEC")) %>% 
  mutate_at(Grouping_Vars, ~str_replace_all(., pattern = "Lq", replacement = "LQ")) %>% 
  mutate_at(Grouping_Vars, ~str_replace_all(., pattern = "Uq", replacement = "UQ"))
```

## Give Unknown Values a standardized value
```{r}
# Specify Different Names for Unknowns
Unknown_types <- c("Unreported", 
                   "Not Stated", 
                   "Not Known") # add refused?

# for each unknown type, look for it in the dataset, and if present, rename to Unknown
for (i in 1:length(Unknown_types)){
    
  if (Unknown_types[i] %in% PreP_Data$Ethnicity){
      print(paste0("Changed: ", Unknown_types[i],  " to Unknown"))
      PreP_Data$Ethnicity[PreP_Data$Ethnicity == Unknown_types[i]] <- "Unknown"
  } else{
    print(paste0("No Instances of: ", Unknown_types[i]))
  }
}

print("Unique Ethnicities: ")
cat(sort(unique(PreP_Data$Ethnicity)), sep = ", ")
```


## Check for Ethnicity names against pre-defined list (White, White British etc.)
```{r Identify Irregular Ethnicities}
# Define Standard List of Ethnicities
Standard_Ethnicity_Categories <- c("All",
                                   "Asian", "Black", "Mixed", # ONS 5+1 
                                   "Other", "White", "Unknown", 
                                   "Indian", "Pakistani", "Bangladeshi", # ONS 18+1 Asian
                                   "Chinese", "Any Other Asian Background", 
                                   "Black Caribbean", "Black African", # ONS 18+1 Black 
                                   "Any Other Black Background",  
                                   "Mixed White and Black Caribbean", # ONS 18+1 Mixed
                                   "Mixed White and Black African",
                                   "Mixed White and Asian",
                                   "Any Other Mixed Or Multiple Ethnic Background", 
                                   "White British", "White Irish", # ONS 18+1 White
                                   "Gypsy Or Irish Traveller", 
                                   "Any Other White Background", 
                                   "Arab", # ONS 18+1 Others
                                   "Any Other Ethnic Group") 

# Recode Common Variations
PreP_Data$Ethnicity <-  recode(PreP_Data$Ethnicity, 
                              "Total" = "All",
                              "African" = "Black African",
                              "Caribbean" = "Black Caribbean", 
                              "Asian Other" = "Any Other Asian Background", 
                              "Black Other" = "Any Other Black Background",
                              "Mixed Other" = "Any Other Mixed/Multiple Ethnic Background", 
                              "White Other" = "Any Other White Background",
                              "White and Black Caribbean" = "Mixed White and Black Caribbean",
                              "White and Black African" = "Mixed White and Black African",
                              "White and Asian" = "Mixed White and Asian") 
                              # add any new renames as:
                              # "BAD_EXAMPLE" = "Good_Example"

# Get Unique Ethnicities in Dataset
Ethnicities_Unique <- unique(PreP_Data$Ethnicity)

  # Flag up any ethnicities that don't match 
if (length(Ethnicities_Unique[!(Ethnicities_Unique %in% Standard_Ethnicity_Categories)]) > 0){
  Atypical_Eths <- Ethnicities_Unique[!(Ethnicities_Unique %in% Standard_Ethnicity_Categories)]
  write_tsv(x = tibble(Atypical_Eths), file = paste("Output/ATYPICAL-ETHNICITIES", Measure_Name, Update_Year, sep="-"))
  print("NOT Located Under Standardised Categories:")
  print(paste(Ethnicities_Unique[!(Ethnicities_Unique %in% Standard_Ethnicity_Categories)], collapse = ", "))
}
```

## Numeric Transformation
Remove any "," "-" values
Transform any "N/A" values to NA
Replace any "!" "c" or "?" values
```{r Treat Numeric Columns}
# Pre-numeric transformation:
# Go through all numeric columns and then replace/remove problematic characters 
PreP_Data <- PreP_Data %>% 
  mutate_at(Numeric_Vars, ~str_replace_all(., pattern = ",", replacement = "")) %>% 
  mutate_at(Numeric_Vars, ~str_replace_all(., pattern = "-", replacement = hyphen_replacement)) %>%  # Hyphen to be defined by analyst
  mutate_at(Numeric_Vars, ~str_replace_all(., pattern = "N/A", replacement = NA_character_)) %>% 
  mutate_at(Numeric_Vars, ~str_replace_all(., pattern = "!", replacement = "-999")) %>% 
  mutate_at(Numeric_Vars, ~str_replace_all(., pattern = "c", replacement = "-998")) %>% 
  mutate_at(Numeric_Vars, ~str_replace_all(., pattern = "\\?", replacement = "-997")) # Need to use '\\' to escape Regex
```

## Make numeric columns truly numeric so we can do operations
```{r Transform to Numeric}

# Count NAs pre-transform
pre_NAs <- PreP_Data %>% 
  select(Numeric_Vars) %>% 
  map_df(., ~sum(is.na(.))) %>% 
  mutate(Condition = "Pre")

# Make numeric columns numeric in dataset
PreP_Data <- PreP_Data %>% 
  mutate_at(Numeric_Vars, ~as.numeric(.))

# Count NAs post-transform
post_NAs <- PreP_Data %>% 
  select(Numeric_Vars) %>% 
  map_df(., ~sum(is.na(.))) %>% 
  mutate(Condition = "Post")

# Combine pre and post NA counts for comparison
Comparison_NAs <- rbind(pre_NAs, post_NAs) %>% 
  pivot_longer(!Condition) 

# drop raw counts
rm(list = c("pre_NAs", "post_NAs"))

# make container df
Comparison_Output <- tibble()

# go through each name (column) and test the pre and post NAs counts...
for (i in 1:length(unique(Comparison_NAs$name))){
  temp_name <- unique(Comparison_NAs$name)[i]
  temp_df <- Comparison_NAs %>% 
    filter(name == temp_name) %>% 
    pivot_wider(names_from = Condition, values_from = value) %>% 
    mutate(Difference_Pre_Post = Post - Pre)
  
  # ...if Difference is not 0 then flag up
  if (temp_df$Difference_Pre_Post[1] != 0){
    beepr::beep()
    print(paste("NA values have increased with Numeric Transformation! Difference: ", temp_df$Difference_Pre_Post[1]))
  }
  
  # Store counts in Container df
  Comparison_Output <- rbind(Comparison_Output, temp_df)
}


Comparison_Output <- Comparison_Output %>% 
  arrange(desc(Difference_Pre_Post)) %>% 
  mutate(Match = if_else(Difference_Pre_Post == 0, "Match", "No Match"))

# drop stuff we don't need
rm(list = c("i", "temp_df", "Comparison_NAs"))



NA_Comp_Table <- reactable(Comparison_Output, 
           highlight = TRUE,
          resizable = TRUE, 
          borderless = FALSE, 
          wrap = TRUE, 
          striped = TRUE, 
    theme = reactableTheme(
    borderColor = "#dfe2e5",
    stripedColor = alpha("#f6f8fa", 5/8),
    highlightColor = alpha("#008080", 3/8), 
    style = list(fontFamily = "-apple-system, BlinkMacSystemFont, Segoe UI, Helvetica, Arial, sans-serif"),
    searchInputStyle = list(width = "100%")), 
    columns = list(
      "name" = colDef(name = "Variable Name", align = "left", style = list(fontWeight = "bold")),
      "Pre" = colDef(name = "Pre NAs", width = 90, align = "right"),
      "Post" = colDef(name = "Post NAs", width = 100, align = "right"),
      "Difference_Pre_Post" = colDef(name = "Difference", width = 120, align = "right"),
      "Match" = colDef(cell = function(Match) {
    # Render as an X  or tick using unicode values
    if (Match == "No Match") "\u274c" else "\u2705"}, width = 90, align = "center")))

NA_Comp_Table
```

```{r}
write_csv(x = PreP_Data, file = paste("Output/(1.) Pre-processed files/Pre-", Measure_Name, "-", Update_Year, ".csv", sep=""))
write_tsv(x = tibble(Grouping_Vars), file = paste0("Output/(5.) Misc/", Measure_Name, "-", Update_Year, "-", "Grouping-Variables.txt"))
write_tsv(x = tibble(Numeric_Vars), file = paste0("Output/(5.) Misc/", Measure_Name, "-", Update_Year, "-", "Grouping-Variables.txt"))
write_tsv(x = tibble(Time_Vars), file = paste0("Output/(5.) Misc/", Measure_Name, "-", Update_Year, "-", "Time-Variables.txt"))
write_tsv(x = tibble(Measure_Name), file = paste0("Output/(5.) Misc/", Measure_Name, "-", Update_Year, "-", "Measure-Name.txt"))
write_tsv(x = tibble(Update_Year), file = paste0("Output/(5.) Misc/", Measure_Name, "-", Update_Year, "-", "Measure-Year.txt"))
htmlwidgets::saveWidget(NA_Comp_Table, file = "NA_Count_Table.html", title = "Number of NAs in Data", selfcontained = TRUE) # file doesn't save properly as self-contained
file.rename(from = "NA_Count_Table.html", to = "Output/(1.) Pre-processed files/NA_Count_Table.html") # so needs to be moved from wd to do so
rm(list = c("NA_Comp_Table"))
print("Pre-processing Complete")
```
'''



robjects.r(r_script)





<rpy2.rinterface_lib.sexp.NULLType object at 0x10c004100> [RTYPES.NILSXP]